In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torch.nn.functional import normalize
from tqdm import tqdm

c:\Python39\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
def read_data(file):
    return pd.read_csv(file, delimiter=',')

In [4]:
def split_data(data):
    x = data[:, :5]
    y = data[:, 5]
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=37)
    return x_train, x_test, y_train, y_test

def transform(data):
    data = torch.tensor(data, dtype=torch.float32)
    # data = normalize(data)
    return data

In [5]:
def preprocess(filename):
    df_raw = read_data(filename)
    print(df_raw.head())
    d_raw = df_raw.to_numpy() # data to binary?
    x, x_test, y, y_test = split_data(d_raw)

    x, x_test = transform(x), transform(x_test)
    
    return x, x_test, y, y_test

In [6]:
datafile = '../data/basic_strategy_agent.csv'
x_train, x_test, y_train, y_test = preprocess(datafile)

   Player sum  Dealer card  Usable ace  Double down allowed  Split allowed  \
0          15            9           0                    1              0   
1          19           10           0                    1              0   
2          11           10           0                    1              0   
3          20           10           0                    1              1   
4          20            9           0                    1              1   

   Action  
0       1  
1       0  
2       2  
3       0  
4       0  


In [7]:
def one_hot_encode(y, n):
    return torch.eye(n)[y]

def combine_input_and_label(x, y, n):
    y_one_hot = one_hot_encode(y, n)
    return torch.concat((x, y_one_hot), 1)

def generate_neg_labels(y, n):
    y_ = y.copy()
    for i in range(len(y)):
        negative_labels = [j for j in range(n) if j != y[i]]
        y_[i] = np.random.choice(negative_labels)

    return y_

In [17]:
x_pos = combine_input_and_label(x_train, y_train, 4)

y_neg = generate_neg_labels(y_train, 4)
x_neg = combine_input_and_label(x_train, y_neg, 4)

y_train_OH = one_hot_encode(y_train, 4)
y_test_OH = one_hot_encode(y_test, 4)


print(f"x_pos[0]: {x_pos[0]}")
print(f"x_neg[0]: {x_neg[0]}")
print(f"Train size: {len(x_train)}")
print(f"Test size: {len(x_test)}")

x_pos[0]: tensor([19.,  3.,  1.,  0.,  0.,  1.,  0.,  0.,  0.])
x_neg[0]: tensor([19.,  3.,  1.,  0.,  0.,  0.,  0.,  1.,  0.])
Train size: 1197
Test size: 300


## Forward-Forward

In [1]:
from FFNN import FFNN
import torch

# Hyperparameters
layers = [9, 2000, 2000, 2000, 2000]
bias = True
threshold = 0.8
epochs = 60
learning_rate = 0.03

model = FFNN(
    layers=layers,
    bias=bias,
    threshold=threshold,
    epochs=epochs,
    learning_rate=learning_rate,
    )

if torch.cuda.is_available():
    device = torch.device('cuda')
    print("using gpu: ", torch.cuda.get_device_name())
else:
    device = torch.device('cpu')
    print("using cpu")

model.to(device)

# x = torch.Tensor([[1,1,5,2,0], [1,1,2,2,1]])
# print(x.shape)
# print(x / torch.sqrt(torch.sum(x**2, dim=1, keepdim=True)))
# print(x / (x.norm(2, 1, keepdim=True)))
# print(x / (torch.sqrt(torch.mean(x ** 2, dim=-1, keepdim=True))))

using cpu


In [11]:
model.train(x_pos.to(device), x_neg.to(device))
torch.save(model.state_dict(), 'model.pth')

Training Layer: 1


100%|██████████| 60/60 [00:00<00:00, 411.37it/s]


Avg Loss: 12.68347699244817
Training Layer: 2


100%|██████████| 60/60 [00:02<00:00, 28.09it/s]


Avg Loss: 4.990047407150269
Training Layer: 3


100%|██████████| 60/60 [00:02<00:00, 27.17it/s]


Avg Loss: 4.211366581916809
Training Layer: 4


100%|██████████| 60/60 [00:02<00:00, 26.38it/s]

Avg Loss: nan


In [12]:
print(x_test)
print(y_test)

model.predict_accumulated_goodness(x_test)

tensor([[15., 10.,  0.,  0.,  0.]])
[1]


tensor([[0]])

In [13]:
test_error = 1 - model.predict_accumulated_goodness(x_test).eq(torch.from_numpy(y_test)).float().mean().item() # heet dit accuracy?
print(test_error)

1.0


## Backpropegation

In [26]:
def train(model, x, y, epochs, loss_function, optimizer, batch_size):
    
    model.train()
    num_batches = len(x) // batch_size
    
    for epoch in range(epochs):
        losses = []
        for batch in tqdm(range(num_batches)):
            start_idx = batch * batch_size
            end_idx = start_idx + batch_size
            
            x_batch = x[start_idx:end_idx]
            y_batch = y[start_idx:end_idx]
            
            predictions = model(x_batch)
            loss = loss_function(predictions, y_batch)
            losses.append(loss.item())
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            
        print(f"epoch: {epoch + 1}/{epochs} avg. loss: {sum(losses)/len(losses)}")

In [27]:
def test(model, x_test, y_test, loss_function):
    
    model.eval()
    
    with torch.no_grad():
        predictions = model(x_test)
        test_loss = loss_function(predictions, y_test).item()
        
    print(f"avg. test loss: {test_loss}")   
    
def predict_action(model, x):
    model.eval()
    
    with torch.no_grad():
        predictions = model(x)
        return predictions.argmax(1)

In [28]:
from backpropNN import BackpropNN

input_size = 5
output_size = 4

# hyperparameters
hidden_size = 30
activation_function = nn.ReLU()

# fully connected neural network with 4 hidden layers
backprop_model = BackpropNN(input_size, output_size, hidden_size, activation_function)

In [32]:
# hyperparameters
epochs = 100
batch_size = 64
learning_rate = 0.001

optimizer = torch.optim.Adam(backprop_model.parameters(), lr=learning_rate);
loss_fn = nn.CrossEntropyLoss()

train(backprop_model, x_train, y_train_OH, epochs, loss_fn, optimizer, batch_size)

100%|██████████| 26/26 [00:00<00:00, 519.77it/s]


epoch: 1/10 avg. loss: 1.3648410806289086


100%|██████████| 26/26 [00:00<00:00, 806.64it/s]


epoch: 2/10 avg. loss: 1.268617909688216


100%|██████████| 26/26 [00:00<00:00, 812.50it/s]


...

epoch: 8/10 avg. loss: 0.970029585636579


100%|██████████| 26/26 [00:00<00:00, 837.89it/s]


epoch: 9/10 avg. loss: 0.9461542574258951


100%|██████████| 26/26 [00:00<00:00, 896.49it/s]


epoch: 10/10 avg. loss: 0.9243824871686789


In [152]:
test(backprop_model, x_test, y_test_OH, loss_fn)

avg. test loss: 0.9507748484611511


In [153]:
# print(torch.from_numpy(y_test))
# print(predict_action(backprop_model, x_test))
accuracy = 1 - predict_action(backprop_model, x_test).eq(torch.from_numpy(y_test)).float().mean().item() # heet dit accuracy?
print(accuracy)

tensor([0, 1, 1,  ..., 1, 0, 0])
tensor([0, 0, 0,  ..., 0, 0, 0])
0.5006591975688934
